In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))
print(os.listdir("../input/training/training"))

# Any results you write to the current directory are saved as output.

In [ ]:
# these seeds are both required for reproducibility
import numpy as np
np.random.seed(42)
import tensorflow as tf
tf.set_random_seed(42)

from keras.applications.inception_v3 import InceptionV3
from keras.models import Model
from keras.layers import Dense, Conv2D, BatchNormalization, MaxPooling2D, Flatten, Dropout, Input
from keras.callbacks import TensorBoard, ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD
import os

In [ ]:
class Configuration:
    def __init__(self):
        self.feature_extraction_epochs = 10
        self.fine_tuning_epochs = 20
        self.epochs_without_transfer_learning = 100
        self.batch_size = 30
        self.data_dir = "../input/training/training"
        self.val_dir = "../input/validation/validation"

In [ ]:
def build_model():
    inputs = Input(shape=(299,299,3), name="input")
    
    #YOUR CODE HERE
    #Network Architecture should be
   
    #Conv2D 128 units, 3x3 kernel, relu activation
    #MaxPooling2D 2x2 pool size
    conv1 = Conv2D(128, kernel_size=(3,3), activation="relu", name="conv_1")(inputs)
    #batch1 = BatchNormalization(name="batch_norm_1")(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2), name="pool_1")(conv1)

    #Conv2D 64 units, 3x3 kernel, relu activation
    #MaxPooling2D 2x2 pool size
    conv2 = Conv2D(64, kernel_size=(3,3), activation="relu", name="conv_2")(pool1)
    #batch2 = BatchNormalization(name="batch_norm_2")(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2), name="pool_2")(conv2)

    #Conv2D 32 units, 3x3 kernel, relu activation
    #MaxPooling2D 2x2 pool size
    conv3 = Conv2D(32, kernel_size=(3,3), activation="relu", name="conv_3")(pool2)
    #batch3 = BatchNormalization(name="batch_norm_3")(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2), name="pool_3")(conv3)

    #Conv2D 16 units, 3x3 kernel, relu activation
    #MaxPooling2D 2x2 pool size
    conv4 = Conv2D(16, kernel_size=(3,3), activation="relu", name="conv_4")(pool3)
    #batch4 = BatchNormalization(name="batch_norm_4")(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2), name="pool_4")(conv4)
    
    # fully connected layers
    # output shows "fc1 (Dense) (None, 1024) 4195328"
    
    flatten = Flatten()(pool4)    
    fc1 = Dense(1024, activation="relu", name="fc1")(flatten)
    
    #fc1 = Dense(1024, activation="relu", name="fc1")(pool4)
    
    #d1 = Dropout(rate=0.2, name="dropout1")(fc1)
    #fc2 = Dense(256, activation="relu", name="fc2")(d1)
    #d2 = Dropout(rate=0.2, name="dropout2")(fc2)
    
    #Dense 1024
    #Dense softmax layer (named "output")
    output = Dense(10, activation="softmax", name="output")(fc1)
    
    

    # finalize and compile
    model = Model(inputs=inputs, outputs=output)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=["accuracy"])
    return model

In [ ]:
def create_callbacks(name):
    tensorboard_callback = TensorBoard(log_dir=os.path.join(os.getcwd(), "tensorboard_log", name), write_graph=True, write_grads=False)
    checkpoint_callback = ModelCheckpoint(filepath="./model-weights-" + name + ".{epoch:02d}-{val_loss:.6f}.hdf5", monitor='val_loss',
                                          verbose=0, save_best_only=True)
    return [tensorboard_callback]

In [ ]:
def setup_data(train_data_dir, val_data_dir, img_width=299, img_height=299, batch_size=16):
    train_datagen = ImageDataGenerator(rescale=1./255)
    val_datagen = ImageDataGenerator(rescale=1./255)

    train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='categorical')

    validation_generator = val_datagen.flow_from_directory(
        val_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='categorical')
    return train_generator, validation_generator

In [ ]:
def fit_model(model, train_generator, val_generator, batch_size, epochs, name):
    model.fit_generator(
        train_generator,
        steps_per_epoch=train_generator.n // batch_size,
        epochs=epochs,
        validation_data=val_generator,
        validation_steps=val_generator.n // batch_size,
        callbacks=create_callbacks(name=name),
        verbose=1)
    return model

In [ ]:
def eval_model(model, val_generator, batch_size):
    scores = model.evaluate_generator(val_generator, steps=val_generator.n // batch_size)
    print("Loss: " + str(scores[0]) + " Accuracy: " + str(scores[1]))

In [ ]:
config = Configuration()

In [ ]:
train_generator, val_generator = setup_data(config.data_dir, config.val_dir, batch_size=config.batch_size)

In [ ]:
model = build_model()
print (model.summary())

In [ ]:
model = fit_model(model, train_generator, val_generator,
                  batch_size=config.batch_size,
                  epochs=config.epochs_without_transfer_learning,
                  name="without_transfer_learning")